In [1]:
import numpy as np
from functions import scales_mean_matrix_factorization, ALSMatrixFactorization


# make user-item matrix (calculating different "rating" metrics)

# do matrix factorization (using different matrix factorization methods: Alternating Least Squares, Scale's Mean, others??)
ratings = np.array([[0.5, 0, 4],
                    [1, 3, 5]])


res = scales_mean_matrix_factorization(ratings)
print("res", res)


# ALSmodel = ALSMatrixFactorization(5, 3, 0.5, ratings)
# res = ALSmodel.fit()
# print(res)

# ALSmodel = ALSMatrixFactorization(15, 3, 0.5, ratings)
# res = ALSmodel.fit()
# print(res)

ALSmodel = ALSMatrixFactorization(5, 3, 0.5, ratings)
res = ALSmodel.fit()
mse = ALSmodel.get_training_mse()
print(mse)
mape = ALSmodel.get_training_mape()
print(mape)




Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
ratings [[0.5 0.  4. ]
 [1.  3.  5. ]]
res [[-2.  -2.5  1.5]
 [-1.5  0.5  2.5]]
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.